In [1]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier()

In [3]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [4]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

In [5]:
import time

s=time.time()
clr.predict(X_test)
e=time.time()
t1=e-s

s=time.time()
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]
e=time.time()
t2=e-s

print("Czas predykcji modelu: ", t1)
print("Czas predykcji onnx: ", t2)
print("Stosunek czasu predykcji modelu względem onnx: ", t1/t2)

Czas predykcji modelu:  0.008885622024536133
Czas predykcji onnx:  0.00099945068359375
Stosunek czasu predykcji modelu względem onnx:  8.89050572519084


In [6]:
import numpy as np
x=np.array([X_test[0]])

In [12]:
import time

s=time.time()
clr.predict(x)
e=time.time()
t1=e-s

s=time.time()
pred_onx = sess.run([label_name], {input_name: x.astype(numpy.float32)})[0]
e=time.time()
t2=e-s

print("Czas predykcji modelu: ", t1)
print("Czas predykcji onnx: ", t2)
print("Stosunek czasu predykcji modelu względem onnx: ", t1/t2)

Czas predykcji modelu:  0.00800013542175293
Czas predykcji onnx:  0.0


ZeroDivisionError: float division by zero